In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [20]:
import numpy as np

In [3]:
df_jan = pd.read_parquet('/home/saiteja/mlops-zoomcamp/Home-work/week-1/fhv_tripdata_2021-01.parquet')
len(df_jan)

1154112

In [4]:
df_feb = pd.read_parquet('/home/saiteja/mlops-zoomcamp/Home-work/week-1/fhv_tripdata_2021-02.parquet')
len(df_feb)

1037692

In [5]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [6]:
df_jan['duration'] = df_jan.dropOff_datetime - df_jan.pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb['duration'] = df_feb.dropOff_datetime - df_feb.pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
df_jan.duration.mean(), df_jan.duration.mean(skipna=False)

(19.1672240937939, 19.167224093791006)

In [23]:
df_jan_trf = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb_trf = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
len(df_jan_trf),len(df_feb_trf) 

(1109826, 990113)

In [24]:
len(df_jan),len(df_feb) 

(1154112, 1037692)

In [25]:
Jan_record_drop=len(df_jan)-len(df_jan_trf)
Feb_record_drop=len(df_feb)-len(df_feb_trf)
Jan_record_drop , Feb_record_drop

(44286, 47579)

In [26]:
categorical = ['PUlocationID', 'DOlocationID']

In [27]:
#df_jan_trf[categorical] = df_jan_trf[categorical].astype(str)
#df_feb_trf[categorical] = df_feb_trf[categorical].astype(str)

In [28]:
df_jan_trf['PUlocationID'] = df_jan_trf['PUlocationID'].replace(np.nan, "-1")
df_jan_trf['DOlocationID'] = df_jan_trf['DOlocationID'].replace(np.nan,  "-1")
df_feb_trf['PUlocationID'] = df_feb_trf['PUlocationID'].replace(np.nan, "-1")
df_feb_trf['DOlocationID'] = df_feb_trf['DOlocationID'].replace(np.nan,  "-1")

/tmp/ipykernel_2093/1138616216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_trf['PUlocationID'] = df_jan_trf['PUlocationID'].replace(np.nan, "-1")
/tmp/ipykernel_2093/1138616216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_trf['DOlocationID'] = df_jan_trf['DOlocationID'].replace(np.nan,  "-1")
/tmp/ipykernel_2093/1138616216.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [30]:
df_jan_trf[categorical] = df_jan_trf[categorical].astype(str)
df_feb_trf[categorical] = df_feb_trf[categorical].astype(str)

/tmp/ipykernel_2093/3767876592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_trf[categorical] = df_jan_trf[categorical].astype(str)
/tmp/ipykernel_2093/3767876592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_trf[categorical] = df_feb_trf[categorical].astype(str)


In [31]:
df_jan_trf.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1,-1,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1,-1,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1,71.0,None,B00037,9.050000


In [38]:
df_jan_trf['PUlocationID'].value_counts(normalize=True)[:2]*100

-1       83.527328
221.0     0.750568
Name: PUlocationID, dtype: float64

In [39]:
df_jan_trf['PU_DO'] = df_jan_trf['PUlocationID'] + '_' + df_jan_trf['DOlocationID']
df_feb_trf['PU_DO'] = df_feb_trf['PUlocationID'] + '_' + df_feb_trf['DOlocationID']

/tmp/ipykernel_2093/678166231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_trf['PU_DO'] = df_jan_trf['PUlocationID'] + '_' + df_jan_trf['DOlocationID']
/tmp/ipykernel_2093/678166231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_trf['PU_DO'] = df_feb_trf['PUlocationID'] + '_' + df_feb_trf['DOlocationID']


In [46]:
categorical = ['PUlocationID','DOlocationID'] #'PULocationID', 'DOLocationID']
dv = DictVectorizer()

train_dicts = df_jan_trf[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_feb_trf[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [47]:
X_train.shape

(1109826, 525)

In [48]:
target = 'duration'
y_train = df_jan_trf[target].values
y_val = df_feb_trf[target].values

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107213159

In [51]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.01428321291995